<a href="https://colab.research.google.com/github/Deepak23mca10060/LanguageTranslator/blob/main/Copy_of_hinglish_translator_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/stokome/Hinglish-Translation-AI-llama2.git

Cloning into 'Hinglish-Translation-AI-llama2'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 58 (delta 25), reused 41 (delta 11), pack-reused 0
Receiving objects: 100% (58/58), 74.17 KiB | 566.00 KiB/s, done.
Resolving deltas: 100% (25/25), done.
Error downloading object: llama-2-7b-hinglish_weights/custom_dataset_weights/adapter_model.bin (ab54b23): Smudge error: Error downloading llama-2-7b-hinglish_weights/custom_dataset_weights/adapter_model.bin (ab54b2390fdcb3d048422fa7cf8996cc87cde8930c19dc809814e80dea0b945e): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/Hinglish-Translation-AI-llama2/.git/lfs/logs/20240601T025123.017758465.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: llama-2-7b-hinglish_weig

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from datasets import Dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
# LLAMA-2 model
model_name = "meta-llama/Meta-Llama-3-8B"

# Fine-tuned model name
new_model = "/content/Hinglish-Translation-AI-llama2/llama-2-7b-hinglish_weights/custom_dataset_weights"

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
!pip install huggingface_hub

# Log in to the Hugging Face model hub
!huggingface-cli login
#after the excution of this program there is asking tokenize key where you go to the Hugging face website and create token for the requried model and paste it over there


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model for translation
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForSeq2SeqLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

# Translation function
def translate_to_hindi(english_text, max_length=64):
    translation_pipeline = pipeline(task="translation_en_to_hi", model=model, tokenizer=tokenizer)
    result = translation_pipeline(english_text, max_length=max_length)
    return result[0]['translation_text']

# Example usage
english_text = "Hello, how are you?"
hindi_translation = translate_to_hindi(english_text)
print(hindi_translation)


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.LlamaConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [ ]:
import re
def translate_to_hinglish(query, max_length=64):
    non_english_chars_pattern = re.compile(r'[^a-zA-Z]+')
    system_prompt = "Convert English to Hinglish"
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)
    result = pipe(f"<s>[INST] <<SYS>> {system_prompt} <</SYS>>{query}[/INST]")
    result = result[0]['generated_text'].split('[/INST]')[1].split('।')[0]+ "।"
    return result

In [ ]:
queries = ["Definitely share your feedback in the comment section.",
         "I was waiting for my bag.",
         "So even if it's a big video, I will clearly mention all the products."]

In [ ]:
for query in queries:
    print(f"English: {query}\n Hinglish: {translate_to_hinglish(query, max_length=100)}\n")

NameError: name 'model' is not defined

In [ ]:
queries = ["My name is Yatharth Anand.",
           "We are going to unbox new Iphone 20.",
           "The sponsor of this video is Kurkure.",
           "I am vlogging in the streets of Barcelone.",
           "Please like and subscribe my youtube channel."]

In [ ]:
for query in queries:
    print(f"English: {query}\n Hinglish: {translate_to_hinglish(query, max_length=100)}\n")

English: My name is Yatharth Anand.
 Hinglish: मेरा नाम Yatharth Anand है।

English: We are going to unbox new Iphone 20.
 Hinglish: हम नए Iphone 20 के unboxing करने जा रहे हैं।

English: The sponsor of this video is Kurkure.
 Hinglish: यह video के sponsor Kurkure है।

English: I am vlogging in the streets of Barcelone.
 Hinglish: मैं बर्सलोन के सड़कों पर vlogging कर रहा हूँ।

English: Please like and subscribe my youtube channel.
 Hinglish: कृपया मेरे youtube channel को like और subscribe करें।



In [ ]:
print("hi")

hi
